Candle Stick coding goo brr


In [24]:
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import numpy as np

pyo.init_notebook_mode(connected=True)

pd.options.plotting.backend = 'plotly'

In [25]:

date_end = dt.datetime.now()
date_start = dt.datetime(2015,1,1)
ticker_symbol = 'AAPL'
stock_data = yf.Ticker(ticker_symbol)
df = stock_data.history(start = date_start, end = date_end)
df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-04-22 00:00:00-04:00,165.520004,167.259995,164.770004,165.839996,48116400,0.0,0.0
2024-04-23 00:00:00-04:00,165.350006,167.050003,164.919998,166.899994,49537800,0.0,0.0
2024-04-24 00:00:00-04:00,166.539993,169.300003,166.210007,169.020004,48251800,0.0,0.0
2024-04-25 00:00:00-04:00,169.529999,170.610001,168.149994,169.889999,50558300,0.0,0.0
2024-04-26 00:00:00-04:00,169.880005,171.339996,169.179993,169.300003,44525100,0.0,0.0


Calculating moving averages below

In [26]:

df['MA50'] = df['Close'].rolling(window=50, min_periods=0).mean()
df['MA200'] = df['Close'].rolling(window=200, min_periods=0).mean()
df['MA200'].head(30)

Date
2015-01-02 00:00:00-05:00    24.435265
2015-01-05 00:00:00-05:00    24.091077
2015-01-06 00:00:00-05:00    23.977090
2015-01-07 00:00:00-05:00    24.003352
2015-01-08 00:00:00-05:00    24.204167
2015-01-09 00:00:00-05:00    24.342514
2015-01-12 00:00:00-05:00    24.353210
2015-01-13 00:00:00-05:00    24.388332
2015-01-14 00:00:00-05:00    24.405218
2015-01-15 00:00:00-05:00    24.352125
2015-01-16 00:00:00-05:00    24.291821
2015-01-20 00:00:00-05:00    24.292415
2015-01-21 00:00:00-05:00    24.307186
2015-01-22 00:00:00-05:00    24.365345
2015-01-23 00:00:00-05:00    24.424391
2015-01-26 00:00:00-05:00    24.477733
2015-01-27 00:00:00-05:00    24.472737
2015-01-28 00:00:00-05:00    24.544907
2015-01-29 00:00:00-05:00    24.651710
2015-01-30 00:00:00-05:00    24.728388
2015-02-02 00:00:00-05:00    24.813409
2015-02-03 00:00:00-05:00    24.890904
2015-02-04 00:00:00-05:00    24.970503
2015-02-05 00:00:00-05:00    25.051415
2015-02-06 00:00:00-05:00    25.116789
2015-02-09 00:00:00-

In [27]:
fig = make_subplots(rows = 2, cols =1, shared_xaxes=True,
                    vertical_spacing = 0.2, subplot_titles=('AAPL', 'Volume'),
                    row_width = [0.2,0.6])

fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'],low = df['Low'], close = df['Close'],name=ticker_symbol),row =1, col=1)
fig.add_trace(go.Bar(x=df.index, y=df['Volume'], marker_color='red', showlegend=False), row=2, col=1)



In [28]:
df['Close']

Date
2015-01-02 00:00:00-05:00     24.435265
2015-01-05 00:00:00-05:00     23.746889
2015-01-06 00:00:00-05:00     23.749117
2015-01-07 00:00:00-05:00     24.082136
2015-01-08 00:00:00-05:00     25.007429
                                ...    
2024-04-22 00:00:00-04:00    165.839996
2024-04-23 00:00:00-04:00    166.899994
2024-04-24 00:00:00-04:00    169.020004
2024-04-25 00:00:00-04:00    169.889999
2024-04-26 00:00:00-04:00    169.300003
Name: Close, Length: 2345, dtype: float64

In [29]:
def green(a,b):
    close = np.array(a)
    open = np.array(b)
    result = close > open
    return result 


def h4_compare(arr):
    n = len(arr)
    sushi_high = np.zeros_like(arr, dtype=bool)  # Create a new array of the same shape as arr
    
    if n < 4:
        return sushi_high
    
    for i in range(n):
        if i < 3:
            sushi_high[i] = False
        else:
            last_two_highs = max(arr[i],arr[i-1])
            front_two_highs = max(arr[i-2],arr[i-3])
            sushi_high[i] = last_two_highs > front_two_highs
    return sushi_high

def l4_compare(arr):
    n = len(arr)
    sushi_low = np.zeros_like(arr, dtype=bool)  # Create a new array of the same shape as arr
    
    if n < 4:
        return sushi_low
    
    for i in range(n):
        if i < 3:
            sushi_low[i] = False
        else:
            last_two_low = max(arr[i],arr[i-1])
            front_two_low = max(arr[i-2],arr[i-3])
            sushi_low[i] = last_two_low < front_two_low
    return sushi_low

def compare_larger(arr):
    # Convert the array to a NumPy array if it is not already
    arr = np.array(arr)

    # Check if the array has at least 2 elements
    if len(arr) < 2:
        raise ValueError("Array must have at least 2 elements.")

    # Compare the nth element with the (n-1)th element for each index n > 0
    comparisons = arr[1:] > arr[:-1]

    # Prepend False as the first element since there is no (n-1)th element for n = 0
    comparisons = np.insert(comparisons, 0, False)

    return comparisons


def compare_smaller(arr):
    # Convert the array to a NumPy array if it is not already
    arr = np.array(arr)

    # Check if the array has at least 2 elements
    if len(arr) < 2:
        raise ValueError("Array must have at least 2 elements.")

    # Compare the nth element with the (n-1)th element for each index n > 0
    comparisons = arr[1:] < arr[:-1]

    # Prepend False as the first element since there is no (n-1)th element for n = 0
    comparisons = np.insert(comparisons, 0, False)

    return comparisons


class momentum(Strategy):
    def init(self):
        self.compare_high = self.I(h4_compare,self.data.High)
        self.compare_low = self.I(l4_compare,self.data.Low)
        self.green = self.I(green,self.data.Close, self.data.Open)

    def next(self):
        if self.compare_low[-1] == True and self.compare_high[-1] == True and self.green[-2] == True:
            self.position.close()
        
        elif self.compare_low[-1] == True and self.compare_high[-1] == True and self.green[-2] == False:
            self.buy()

     #   if len(self.green) >= 2:
      #      if self.green[-1] == 1 and self.green[-2] == 0:
          #  else:
       ##         self.buy()
         #       self.position.close()

        
    


backtest1 = Backtest(df,momentum,commission=0.002, exclusive_orders=True)
backtest1.run()
backtest1.plot()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p1859', ...)

In [30]:
arr = np.array([5, 2, 7, 3, 10])

comparisons = arr[1:] > arr[:-1]
print(comparisons)

[False  True False  True]


In [31]:
import numpy as np

def h4_compare(arr):
    n = len(arr)
    sushi_high = np.zeros_like(arr, dtype=bool)  # Create a new array of the same shape as arr
    
    if n < 4:
        return sushi_high
    
    for i in range(n):
        if i < 3:
            sushi_high[i] = False
        else:
            last_two_highs = max(arr[i],arr[i-1])
            front_two_highs = max(arr[i-2],arr[i-3])
            sushi_high[i] = last_two_highs > front_two_highs
    return sushi_high

def l4_compare(arr):
    n = len(arr)
    sushi_low = np.zeros_like(arr, dtype=bool)  # Create a new array of the same shape as arr
    
    if n < 4:
        return sushi_low
    
    for i in range(n):
        if i < 3:
            sushi_low[i] = False
        else:
            last_two_low = max(arr[i],arr[i-1])
            front_two_low = max(arr[i-2],arr[i-3])
            sushi_low[i] = last_two_low < front_two_low
    return sushi_low

testing = [1,2,3,4,5,2,3]
print(l4_compare(testing))

[False False False False False False  True]
